# Getting Started with stackelberg-opt

This notebook introduces the basic concepts and usage of the stackelberg-opt library for Stackelberg game-theoretic optimization of compound AI systems.

## Table of Contents
1. [Installation](#installation)
2. [Basic Concepts](#basic-concepts)
3. [Creating Modules](#creating-modules)
4. [Running Optimization](#running-optimization)
5. [Analyzing Results](#analyzing-results)

## 1. Installation <a name="installation"></a>

First, install the stackelberg-opt library and its dependencies:

In [ ]:
# Install from the local directory (for development)
!pip install -e ..

# Or install specific dependencies if needed
# !pip install numpy scipy litellm networkx matplotlib

## 2. Basic Concepts <a name="basic-concepts"></a>

stackelberg-opt implements a Stackelberg game-theoretic approach to optimizing compound AI systems:

- **Leader modules**: Make decisions first and influence the system
- **Follower modules**: React to leader decisions
- **Independent modules**: Operate without strategic dependencies

The optimization process uses evolutionary algorithms with bilevel optimization to find equilibrium solutions.

In [ ]:
# Import core components
from stackelberg_opt import (
    Module, 
    ModuleType, 
    SystemCandidate,
    StackelbergOptimizer,
    OptimizerConfig
)

# Import visualization utilities
from stackelberg_opt.utils import OptimizationVisualizer

import asyncio
import numpy as np
from typing import Dict, Tuple

print("stackelberg-opt imported successfully!")

## 3. Creating Modules <a name="creating-modules"></a>

Let's create a simple text summarization system with leader-follower dynamics:

In [ ]:
# Define system modules
modules = {
    # Leader: Extracts key information
    "key_extractor": Module(
        name="key_extractor",
        prompt="""Extract the 3 most important points from this text.
Text: {input_text}

Output the key points as a numbered list:""",
        module_type=ModuleType.LEADER,
        dependencies=[]
    ),
    
    # Follower: Creates summary based on key points
    "summarizer": Module(
        name="summarizer",
        prompt="""Based on these key points, write a concise summary.
Key points:
{key_points}

Summary:""",
        module_type=ModuleType.FOLLOWER,
        dependencies=["key_extractor"]
    ),
    
    # Independent: Adds a title
    "title_generator": Module(
        name="title_generator",
        prompt="""Generate a short, catchy title for this summary.
Summary: {summary}

Title:""",
        module_type=ModuleType.INDEPENDENT,
        dependencies=["summarizer"]
    )
}

print("Created {} modules:".format(len(modules)))
for name, module in modules.items():
    print(f"  - {name} ({module.module_type.value})")

## 4. Running Optimization <a name="running-optimization"></a>

Now let's create a task executor and run the optimization:

In [ ]:
# Create a simple task executor (mock implementation for demo)
async def mock_task_executor(modules: Dict[str, Module], input_text: str) -> Tuple[str, any]:
    """Mock task executor that simulates module execution."""
    from stackelberg_opt import ExecutionTrace
    import time
    import random
    
    trace = ExecutionTrace()
    trace.execution_order = []
    trace.module_outputs = {}
    trace.module_timings = {}
    trace.intermediate_scores = {}
    
    # Simulate execution
    start_time = time.time()
    
    # Execute key_extractor
    key_points = "1. Main idea\n2. Supporting point\n3. Conclusion"
    trace.execution_order.append("key_extractor")
    trace.module_outputs["key_extractor"] = key_points
    trace.module_timings["key_extractor"] = 0.1
    trace.intermediate_scores["key_extractor"] = 0.7 + random.random() * 0.3
    
    # Execute summarizer
    summary = "This text discusses the main idea with supporting evidence."
    trace.execution_order.append("summarizer")
    trace.module_outputs["summarizer"] = summary
    trace.module_timings["summarizer"] = 0.15
    trace.intermediate_scores["summarizer"] = 0.6 + random.random() * 0.3
    
    # Execute title_generator
    title = "Key Insights and Analysis"
    trace.execution_order.append("title_generator")
    trace.module_outputs["title_generator"] = title
    trace.module_timings["title_generator"] = 0.05
    trace.intermediate_scores["title_generator"] = 0.8 + random.random() * 0.2
    
    trace.success = True
    trace.final_score = np.mean(list(trace.intermediate_scores.values()))
    
    final_output = f"{title}\n\n{summary}"
    return final_output, trace

# Create training data
train_data = [
    ("The quick brown fox jumps over the lazy dog. This pangram contains all letters.",
     "Complete Alphabet\n\nThis sentence demonstrates all 26 letters."),
    ("Machine learning transforms data into insights through pattern recognition.",
     "ML Insights\n\nML converts data to knowledge via patterns."),
    ("Climate change requires immediate global action to prevent catastrophe.",
     "Climate Crisis\n\nUrgent worldwide response needed for climate."),
]

print(f"Created {len(train_data)} training examples")

In [ ]:
# Configure the optimizer
config = OptimizerConfig(
    budget=20,  # Small budget for demo
    population_size=5,
    mutation_rate=0.7,
    crossover_rate=0.3,
    performance_weight=0.5,
    equilibrium_weight=0.3,
    stability_weight=0.2,
    enable_visualization=True,
    verbose=True
)

print("Optimizer configuration:")
print(f"  Budget: {config.budget}")
print(f"  Population size: {config.population_size}")
print(f"  Weights: performance={config.performance_weight}, "
      f"equilibrium={config.equilibrium_weight}, "
      f"stability={config.stability_weight}")

In [ ]:
# Create and run the optimizer
optimizer = StackelbergOptimizer(
    system_modules=modules,
    train_data=train_data,
    task_executor=mock_task_executor,
    config=config
)

print("Starting optimization...")
print("=" * 50)

# Run optimization (using asyncio for the async task executor)
best_candidate = await optimizer.optimize_async()

print("\nOptimization complete!")
print(f"Best candidate ID: {best_candidate.candidate_id}")
print(f"Generation: {best_candidate.generation}")
print(f"Average score: {best_candidate.get_average_score():.3f}")

## 5. Analyzing Results <a name="analyzing-results"></a>

Let's analyze the optimization results and visualize the progress:

In [ ]:
# Display optimized prompts
print("Optimized Prompts:")
print("=" * 50)

for module_name, module in best_candidate.modules.items():
    print(f"\n{module_name} ({module.module_type.value}):")
    print("-" * 40)
    print(module.prompt[:200] + "..." if len(module.prompt) > 200 else module.prompt)

# Display scores
print("\n\nPerformance Metrics:")
print("=" * 50)
print(f"Average Score: {best_candidate.get_average_score():.3f}")
print(f"Equilibrium Value: {best_candidate.equilibrium_value:.3f}")
print(f"Stability Score: {best_candidate.stability_score:.3f}")

# Individual run scores
if best_candidate.scores:
    print("\nIndividual Run Scores:")
    for run_id, score in best_candidate.scores.items():
        print(f"  Run {run_id}: {score:.3f}")

In [ ]:
# Create visualization
import matplotlib.pyplot as plt

visualizer = OptimizationVisualizer()

# Plot optimization progress
if hasattr(optimizer, 'population_manager') and optimizer.population_manager.generation_stats:
    fig = visualizer.plot_optimization_progress(optimizer, save=False)
    plt.show()
else:
    print("No generation statistics available for visualization")

In [ ]:
# Test the optimized system
test_text = "Artificial intelligence is revolutionizing healthcare through early disease detection."

print("Testing optimized system:")
print(f"Input: {test_text}")
print("\nOutput:")

output, trace = await mock_task_executor(best_candidate.modules, test_text)
print(output)

print("\nExecution trace:")
for module in trace.execution_order:
    score = trace.intermediate_scores.get(module, 0)
    print(f"  {module}: {score:.3f}")

## Summary

In this notebook, we've covered:

1. **Installation** of the stackelberg-opt library
2. **Basic concepts** of Stackelberg game-theoretic optimization
3. **Creating modules** with leader-follower-independent structure
4. **Running optimization** with custom task executors
5. **Analyzing results** including scores and visualizations

### Next Steps

- Try the advanced examples notebook for more complex systems
- Explore different module configurations and dependencies
- Implement your own task executor for real LLM integration
- Tune optimizer parameters for better performance

For more information, see the [documentation](https://github.com/youraanshshah/stackelberg-opt).